# Otimização:  Método dos Gradiente Conjugados

*Descrição da Tarefa:*

Implementar o método dos *Gradientes Conjugados* (com o Gram-Schmidt completo e só olhando para a última direção) e utilizá-lo para minimizar funções da forma.

$$f(x) = \sum\limits_{i=1}^na_i x_i^2$$

para diferentes valores de $a_i > 0$

### **Funções Auxiliares**

Vamos definir algumas funções que serão utilizadas posteriormente para facilitar os cálculos do *método gradiente*.

In [1]:
import numpy as np
import time

gradient = lambda x,a: 2*x*a # retorna lista com os gradientes dado ponto (x,y)
apply_func = lambda x,a: np.dot(np.square(x), a) # f(x)
close_to_zero = lambda grad, tol: np.linalg.norm(grad,2) < tol # stop condition (euclidian distance)

def gram_schimidt(grad, a, di):
    resp = np.zeros(len(di[0]))
    for d in di:
        resp += np.dot((np.dot(np.multiply(grad,2*a), d)/np.dot(np.multiply(d,2*a), d)),d)
    return resp

### Função Principal

Agora já conseguimos definir a função *conjugated_gradient* (método dos gradientes conjugados). Esta função possui 2 modos de execução:

- **Gram-Schimidt Completo:** Para `complete_schimidt=True`, utilizamos o Gram Schimidt completo (todas as direções anteriores)
- **Apenas última direção:** Para `complete_schimidt=False`, utilizamos apenas a última direção.

In [31]:
def gradient_descent(x, a, complete_schimidt=True, tolerance=0.01, verbose=False, print_d=False):
    
    di = []
    grad = gradient(x,a)
    d = -1*grad
    it = 0

    while((close_to_zero(grad,tolerance) == False)):
        
        if(print_d == True):
            print('it = ', it, ', d = ', d)
            
        if(complete_schimidt == True):
            di.append(d)
        else:
            di = [d]
            
        alpha = -1*(np.dot(grad,d))/(np.dot(np.multiply(d,2*a), d))
        x = x + (d*alpha)
        grad = gradient(x,a)
        d = -1*grad + gram_schimidt(grad, a, di)
        it = it + 1
        
    if(verbose == True):
        print('Iteracoes utilizadas = ', it)
        
    return(x)

## Experimentos

### Diferença das Direções

In [41]:
a = np.float64(range(6)) # temos, por exemplo, esses valores de a > 0
x = np.float64(range(6)) # e um chute inicial dos valores de xi

convergence_x = gradient_descent(x, a, complete_schimidt=True, tolerance=0.001, verbose=True, print_d=True)
print("Ponto de mínimo encontrado: ", convergence_x)

it =  0 , d =  [ -0.  -2.  -8. -18. -32. -50.]
it =  1 , d =  [ 0.         -1.60894864 -4.66585106 -6.51579201 -4.50385624  4.02487152]
it =  2 , d =  [ 0.         -1.19376771 -2.00525116 -0.66959609  1.39155136 -0.42995533]
it =  3 , d =  [ 0.         -0.76359946 -0.37058269  0.49909561 -0.23441844  0.04204368]
it =  4 , d =  [ 0.         -0.36234855  0.18117427 -0.0805219   0.02264678 -0.00289879]
Iteracoes utilizadas =  5
Ponto de mínimo encontrado:  [ 0.00000000e+00 -2.77555756e-17  0.00000000e+00  7.28583860e-17
  1.20563282e-16  3.26670115e-16]


In [42]:
a = np.float64(range(6)) # temos, por exemplo, esses valores de a > 0
x = np.float64(range(6)) # e um chute inicial dos valores de xi

convergence_x = gradient_descent(x, a, complete_schimidt=False, tolerance=0.001, verbose=True, print_d=True)
print("Ponto de mínimo encontrado: ", convergence_x)

it =  0 , d =  [ -0.  -2.  -8. -18. -32. -50.]
it =  1 , d =  [ 0.         -1.60894864 -4.66585106 -6.51579201 -4.50385624  4.02487152]
it =  2 , d =  [ 0.         -1.19376771 -2.00525116 -0.66959609  1.39155136 -0.42995533]
it =  3 , d =  [ 0.         -0.76359946 -0.37058269  0.49909561 -0.23441844  0.04204368]
it =  4 , d =  [ 0.         -0.36234855  0.18117427 -0.0805219   0.02264678 -0.00289879]
Iteracoes utilizadas =  5
Ponto de mínimo encontrado:  [ 0.00000000e+00 -2.77555756e-17 -4.16333634e-17  6.93889390e-18
 -9.71445147e-17 -3.62340366e-16]


**Análise:** Podemos observar que, para este exemplo, as direções encontradas com o gram-schimdt completo e utilizando apenas a última direção são bem similares, resultando na convergência em um mesmo número de iterações. Na seção de análises adicionais mostramos que há contra-exemplos, com configurações específicas dos coeficientes da função (que implicam em diferentes autovalores)

### Comparação das Funções

**i)** $f(x_1, x_2) = 1x_1^2 + 1.1x_2^2$

- Com método completo:

In [6]:
a = np.float64([1, 1.1]) # temos, por exemplo, esses valores de a > 0
x = np.float64([1,2]) # e um chute inicial dos valores de xi

convergence_x = gradient_descent(x, a, complete_schimidt=True, tolerance=0.001, verbose=True)
print("Ponto de mínimo encontrado: ", convergence_x)

Iteracoes utilizadas =  2
Ponto de mínimo encontrado:  [-2.77555756e-17 -4.85722573e-17]


- Apenas última direção:

In [7]:
a = np.float64([1, 1.1]) # temos, por exemplo, esses valores de a > 0
x = np.float64([1,2]) # e um chute inicial dos valores de xi

convergence_x = gradient_descent(x, a, complete_schimidt=False, tolerance=0.001, verbose=True)
print("Ponto de mínimo encontrado: ", convergence_x)

Iteracoes utilizadas =  2
Ponto de mínimo encontrado:  [-2.77555756e-17 -4.85722573e-17]


**Análise:** Podemos observar que ambos os experimentos, neste caso, levam para exatamente o mesmo valor utilizando exatamente o mesmo número de iterações.

**ii)** $f(x_1, x_2) = 1x_1^2 + 100x_2^2$

- Com método completo:

In [8]:
a = np.float64([1, 100]) # temos, por exemplo, esses valores de a > 0
x = np.float64([1,2]) # e um chute inicial dos valores de xi

convergence_x = gradient_descent(x, a, complete_schimidt=True, tolerance=0.001, verbose=True)
print("Ponto de mínimo encontrado: ", convergence_x)

Iteracoes utilizadas =  2
Ponto de mínimo encontrado:  [0.00000000e+00 1.79462565e-16]


- Usando apenas última direção:

In [9]:
a = np.float64([1, 100]) # temos, por exemplo, esses valores de a > 0
x = np.float64([1,2]) # e um chute inicial dos valores de xi

convergence_x = gradient_descent(x, a, complete_schimidt=False, tolerance=0.001, verbose=True)
print("Ponto de mínimo encontrado: ", convergence_x)

Iteracoes utilizadas =  2
Ponto de mínimo encontrado:  [0.00000000e+00 1.79462565e-16]


**Análise:**  Podemos observar que ambos os experimentos, neste caso, levam para exatamente o mesmo valor utilizando exatamente o mesmo número de iterações. Em particular, sabemos que isso ocorre pois, como apenas duas iterações são necessárias, ambos os métodos apenas utilizam uma direção para atualizar os valores e, portanto, para detectar diferenças entre os valores gerados por cada método seria necessário apresentar exemplos que necessitam de mais iterações para convergir.

## Análises Adicionais

É importante comparar a eficiência de cada implementação, apesar de obter iterações semelhantes, pode haver uma diferença no tempo de execução. Em particular, temos a hipótese de que o gram-schimdt completo seria menos computacionalmente eficiente. Para confirmar essa hipótese, conduzimos um experimento simples no qual medimos o tempo de execução em cada um.

**Para Gram-Schimdt Completo:** 

In [10]:
a = np.float64(range(1,100)) # temos, por exemplo, esses valores de a > 0
x = np.float64(range(1,100)) # e um chute inicial dos valores de xi

start_time = time.clock()
convergence_x = gradient_descent(x, a, complete_schimidt=True, tolerance=0.001, verbose=False)
total_time = time.clock() - start_time
print("Tempo de execução de aproximadamente", round(total_time,4), 'segundos')

Tempo de execução de aproximadamente 0.0246 segundos


**Usando apenas o última Direção:** 

In [11]:
a = np.float64(range(1,100)) # temos, por exemplo, esses valores de a > 0
x = np.float64(range(1,100)) # e um chute inicial dos valores de xi

start_time = time.clock()
convergence_x = gradient_descent(x, a, complete_schimidt=False, tolerance=0.001, verbose=False)
total_time = time.clock() - start_time
print("Tempo de execução de aproximadamente", round(total_time,4), 'segundos')

Tempo de execução de aproximadamente 0.0072 segundos


**Análise:** Podemos observar que há uma diferença considerável no tempo de execução entre os métodos. Em particular, nota-se que o método utilizando apenas a última direção é muito mais rápido (0.005s contra os 0.022s do método completo), além de também economizar memória por não precisar armazenar todas as direções calculadas anteriormente.

### Clusters de Autovalores

Na procura de configurações da função que resultassem em alguma diferença de iterações para obter convergência com cada um dos métodos, executamos os seguintes experimentos:
- Gram Schimdt Completo:

In [27]:
a = np.float64([1,1.1,1.2,1.4,1.4,1.3,100000000000]) # temos, por exemplo, esses valores de a > 0
x = np.float64([1,1,1,1,1,1,1]) # e um chute inicial dos valores de xi

convergence_x = gradient_descent(x, a, complete_schimidt=True, tolerance=0.001, verbose=True, print_d=False)
print("Ponto de mínimo encontrado: ", convergence_x)

Iteracoes utilizadas =  5
Ponto de mínimo encontrado:  [ 2.41776204e-05 -7.99260179e-05  1.00740085e-04  6.16776031e-06
  6.16776031e-06 -5.72251371e-05  9.25988880e-22]


- Apenas última direção:

In [44]:
a = np.float64([1,1.1,1.2,1.4,1.4,1.3,100000000000]) # temos, por exemplo, esses valores de a > 0
x = np.float64([1,1,1,1,1,1,1]) # e um chute inicial dos valores de xi

convergence_x = gradient_descent(x, a, complete_schimidt=False, tolerance=0.001, verbose=True, print_d=False)
print("Ponto de mínimo encontrado: ", convergence_x)

Iteracoes utilizadas =  7
Ponto de mínimo encontrado:  [ 2.41776221e-05 -7.99260182e-05  1.00740084e-04  6.16776095e-06
  6.16776095e-06 -5.72251382e-05  4.41631729e-21]


**Análise:** Podemos observar que, ao estabelecer um conjunto de autovalores próximos (cluster) e outro(s) autovalor(es) bem separados, os resultados obtidos pelos dois métodos apresentam certa diferença. Em particular, nota-se que que o método completo é capaz de realizar menos iterações enquanto o método que se utiliza apenas a última direção necessita de mais iterações para convergir. 